### Run ACTION_RECOGNITION.ipynb before running model_analysis.ipynb #####

In [ ]:
from sklearn import metrics
import os
import pickle
import matplotlib.pyplot as plt

### Tuning parameters

In [1]:
# Strength of precision vs recall
beta = 1

NameError: name 'beta' is not defined

### Load model data and generate metrics

In [ ]:
cur_uuid = str(input("cur_uuid:"))
dir_path = f"../pickle/{cur_uuid}"

if not (cur_uuid != "" and os.path.isdir(dir_path)):
    print("ERROR :: UUID not found")
    print("Please enter the correct UUID or run action_recognition.ipynb and pass in the associated UUID")
    exit(1)

print("Loading data...")

with open(dir_path + "/train_test_splits.p", 'rb') as fp:
    training_data = pickle.load(fp)

X_train, X_test = training_data["X_train"], training_data["X_test"]
y_train, y_test = training_data["y_train"], training_data["y_test"]

all_predictions_fns = [file_name for file_name in os.listdir(dir_path) if file_name.startswith("prediction")]

# results_dict[str(action_classifier)] = {
#   "y_pred": y_pred,
#   "action_classifier_params": action_classifier_params,
#   "confusion_matrix": confusion_matrix,
#   "precision": precision,
#   "recall": recall,
#   "fbetascore": fscore,
#   "support": support,
#   "accuracy": accuracy,
#   "prediction_speed": prediction_speed
# }
predictions_dict = {}

for prediction_fn in all_predictions_fns:
    with open(f"{dir_path}/{prediction_fn}.p", 'rb') as fp:
        prediction_info = pickle.load(fp)

        action_classifier_name = prediction_info["action_classifier_name"]
        y_pred = prediction_info["y_pred"]
        action_classifier_params = prediction_info["action_classifier_params"]
        prediction_speed = prediction_info["prediction_speed"]

        confusion_matrix = metrics.confusion_matrix(y_test, y_pred)
        precision, recall, fscore, support = metrics.precision_recall_fscore_support(y_test, y_pred, beta=beta)
        accuracy = metrics.accuracy_score(y_test, y_pred)

        predictions_dict[action_classifier_name] = {
            "y_pred": y_pred,
            "action_classifier_params": action_classifier_params,
            "confusion_matrix": confusion_matrix,
            "precision": precision,
            "recall": recall,
            "fbetascore": fscore,
            "support": support,
            "accuracy": accuracy,
            "prediction_speed": prediction_speed
        }

#### Organize hyper parameter combinations

In [ ]:
# hpc_combinations[hpc_name] = (
#                                 set("str(action_recognition) with wildcard for hpc_value", ... (all hpc name templates)),
#                                 set(hpc_value, ... (all hpc_values))
#                              )
hpc_combinations = {}

for kappa in [10, 50, 100]:
    for n in [50, 100]:
        for theta in [0.3, 0.5, 0.7]:
            for n_neighbors in [10, 50, 100]:
                for n_training_neighbors in [2000, 5000, 20000]:
                    for alpha in [0.75]:
                        for beta in [0.6]:

for model_name, prediction_info in predictions_dict.items():
    action_classifier = prediction_info["action_classifier_params"]
    pose_estimator = action_classifier["nearest_pose_estimator"]

    theta = action_classifier["theta"]
    n = action_classifier["n"]

    alpha = pose_estimator["alpha"]
    beta = pose_estimator["beta"]
    kappa = pose_estimator["kappa"]
    n_neighbors = pose_estimator["n_neighbors"]
    n_training_neighbors = pose_estimator["n_training_neighbors"]

#### Visualize hyperparameter combinations

In [ ]:
plt.scatter()

time_vals = []
for x_val in x_vals:
    total = 0
    for _ in range(3):
        start_time = time.time()
        func(x_val)
        total += time.time() - start_time
    time_vals.append(total/3)

plt.figure()
plt.ylabel("Time (s)")
plt.xlabel("Input (n)")
plt.title(f"Time Complexity of {algorithm_name}")

for x_val, time_val in zip(x_vals, time_vals):
    plt.scatter(len(x_val), time_val)

plt.show()
